# @decorators('are', 'fun')

## Alex Kavanagh
## Canonical

### Python NE January 2016

@ajkavanagh

# What are we covering?

* What IS a decorator?
* Types of decorator
* Why use a decorator
* How to write a decorator
* @memoize
* Summary
* Dojo: StateMachine

# What IS a decorator?

In [1]:
from functools import wraps

def decorator1(f):
    print("I'm decorator 1!")
    return f

def decorator2(prefix):
    def inner(f):
        def wrapped(*args, **kwargs):
            return prefix + " " + f(*args, **kwargs)
        return wraps(f)(wrapped)
    return inner

In [2]:
@decorator1
def function1a():
    """I'm function1a"""
    pass

@decorator2("hello")
def function2a(name):
    """I'm function2a"""
    return name

function1a()
function2a('Gordon')

I'm decorator 1!


'hello Gordon'

In [10]:
def function1b():
    """I'm function1b"""
    pass

function1b = decorator1(function1b)

# and

def function2b(name):
    """I'm function2b"""
    return name

function2b = decorator2('hello')(function2b)

function1b()
function2b('Gordon')

I'm decorator 1!


'hello Gordon'

# Let's write a simple no-argument decorator

Spec: return a JSONified string of the return argument.

In [12]:
import json

def jsonify(f):
    def inner(*args, **kwargs):
        result = f(*args, **kwargs)
        if isinstance(result, dict):
            return json.dumps(result)
        if result is None:
            return None
        raise ValueError("Can't JSONify: {}".format(result))
    return inner

## And try it out ...

In [18]:
@jsonify
def return_json(x):
    """I return {'an': x} when called."""
    return {"an": x}

In [19]:
return_json("apple")

'{"an": "apple"}'

In [20]:
return_json(None)

'{"an": null}'

# But there's a small problem

In [22]:
print(return_json.__name__)
print(return_json.__doc__)

inner
None


## Enter functools.wraps()

* @functools.wraps() --> "This is a convenience function for invoking update_wrapper() as a function decorator when defining a wrapper function. It is equivalent to partial(update_wrapper, wrapped=wrapped, assigned=assigned, updated=updated)."   *https://docs.python.org/3/library/functools.html#functools.update_wrapper*

That was helpful.

Not

# Alright what about update_wrapper() ?

* update_wrapper(...) --> "Update a wrapper function to look like the wrapped function. ..."

Okay, now we're getting somewhere.

In [24]:
def jsonify(f):
    def inner(*args, **kwargs):
        result = f(*args, **kwargs)
        if isinstance(result, dict):
            return json.dumps(result)
        if result is None:
            return None
        raise ValueError("Can't JSONify: {}".format(result))
    return wraps(f)(inner)

In [25]:
@jsonify
def return_json(x):
    """I return {'an': x} when called."""
    return {"an": x}

In [26]:
print(return_json.__name__)
print(return_json.__doc__)

return_json
I return {'an': x} when called.


# Types of Decorator

## Simple function decorator

```python
@decorator
def function(*args, **kwargs):
    ...
```

## Function decorator with arguments

```python
@decorator(arg1, arg2)
def function(...):
    ...
```

## Class decorator (both types)

```python
@class_decorator(arg1, arg2)
class Thing(object):
    ...
```

# Why use a decorator?

* Augment functionality of a function or class
    * e.g. jsonify
    * Memoize
    * instrument (analyse, log)
* Locks and synchronisation (e.g. `@synchronise(some_lock)`)
* Enforcing authentication and authorisation (authn & authz)
* Debugging
* Aspect-orientated programming (https://en.wikipedia.org/wiki/Aspect-oriented_programming)
* Transactions & Rate Limits (e.g. `@ratelimit(minutes=1, requests=10)`)

... and ...

* because it's fun!

# Okay, now lets do @memoize

In [39]:
import time

def timeit(f):
    @wraps(f)
    def wrapped_f(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print("Time taken: {0:.2f}".format(time.time() - start))
        return result
    return wrapped_f

@timeit
def timer(f):
    return f()

@timeit
def check_timeit():
    time.sleep(.5)

check_timeit()

Time taken: 0.50


# Fibonacci

In [48]:
# Using recursion
def fib(n):
    if n == 1 or n == 2:
        return 1
    return fib(n-1)+fib(n-2)


timer(lambda: fib(35))

Time taken: 3.37


9227465

# Memoize

In [54]:
# note that this decorator ignores **kwargs
def memoize(obj):
    cache = obj.cache = {}

    @wraps(obj)
    def memoizer(*args, **kwargs):
        if args not in cache:
            cache[args] = obj(*args, **kwargs)
        return cache[args]
    return memoizer

fib_try = memoize(fib)

In [55]:
timer(lambda: fib_try(35))

Time taken: 3.22


9227465

**hey, why didn't that work?**

In [51]:
@memoize
def fib2(n):
    if n == 1 or n == 2:
        return 1
    return fib2(n-1)+fib2(n-2)

In [53]:
timer(lambda: fib2(35))

Time taken: 0.00


9227465

In [57]:
timer(lambda: fib2(100))

Time taken: 0.00


354224848179261915075

# Another way to write a decorator: with a Class

In [69]:
## Example 5: Using memoization as decorator
class Memoize:
    """Classy Memoize"""
    def __init__(self, fn):
        self.fn = fn
        self.memo = {}
    def __call__(self, arg):
        if arg not in self.memo:
            self.memo[arg] = self.fn(arg)
        return self.memo[arg]
    
@Memoize
def fib3(n):
    """fib3"""
    if n == 1 or n == 2:
        return 1
    return fib3(n-1) + fib3(n-2)

timer(lambda: fib3(100))

Time taken: 0.00


354224848179261915075

In [71]:
print(type(fib3))
print(fib3.__doc__)

### oops

<class '__main__.Memoize'>
Classy Memoize


# Summary

* Everything in Python is an object
* Python 'reads' and executes files
* Functions are first-class objects
* `__name__` and `__doc__` need to be carried over?
* Lot's of examples on the web!

# Coder Dojo (Optional!)

Write a 'StateMachine' class + decorator that allows you to write code like:

```python
stateMachine = StateMachine()

@stateMachine.state('state1', 'action1')
def x(*args):
    pass
    
@stateMachine.state('state1', 'action2')
def y(*args):
    pass
    
@stateMachine.state('state2', 'action1')
def z(*args):
    pass
    
stateMachine.setState('state1')
stateMachine.go('action2', args)  # calls z(*args)
```